# Day 9: Rope Bridge

This rope bridge creaks as you walk along it. You aren't sure how old it is, or whether it can even support your weight.

It seems to support the Elves just fine, though. The bridge spans a gorge which was carved out by the massive river far below you.

You step carefully; as you do, the ropes stretch and twist. You decide to distract yourself by modeling rope physics; maybe you can even figure out where not to step.

Consider a rope with a knot at each end; these knots mark the head and the tail of the rope. If the head moves far enough away from the tail, the tail is pulled toward the head.

Due to nebulous reasoning involving Planck lengths, you should be able to model the positions of the knots on a two-dimensional grid. Then, by following a hypothetical series of motions (your puzzle input) for the head, you can determine how the tail will move.

Due to the aforementioned Planck lengths, the rope must be quite short; in fact, the head (H) and tail (T) must always be touching (diagonally adjacent and even overlapping both count as touching):

```
....
.TH.
....

....
.H..
..T.
....

...
.H. (H covers T)
...
```

If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:

```
.....    .....    .....
.TH.. -> .T.H. -> ..TH.
.....    .....    .....

...    ...    ...
.T.    .T.    ...
.H. -> ... -> .T.
...    .H.    .H.
...    ...    ...
```

Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:

```
.....    .....    .....
.....    ..H..    ..H..
..H.. -> ..... -> ..T..
.T...    .T...    .....
.....    .....    .....

.....    .....    .....
.....    .....    .....
..H.. -> ...H. -> ..TH.
.T...    .T...    .....
.....    .....    .....
```

You just need to work out where the tail goes as the head follows a series of motions. Assume the head and the tail both start at the same position, overlapping.

For example:

```
R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
```

This series of motions moves the head right four steps, then up four steps, then left three steps, then down one step, and so on. After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail. Visually, these motions occur as follows (s marks the starting position as a reference point):

```
== Initial State ==

......
......
......
......
H.....  (H covers T, s)

== R 4 ==

......
......
......
......
TH....  (T covers s)

......
......
......
......
sTH...

......
......
......
......
s.TH..

......
......
......
......
s..TH.

== U 4 ==

......
......
......
....H.
s..T..

......
......
....H.
....T.
s.....

......
....H.
....T.
......
s.....

....H.
....T.
......
......
s.....

== L 3 ==

...H..
....T.
......
......
s.....

..HT..
......
......
......
s.....

.HT...
......
......
......
s.....

== D 1 ==

..T...
.H....
......
......
s.....

== R 4 ==

..T...
..H...
......
......
s.....

..T...
...H..
......
......
s.....

......
...TH.
......
......
s.....

......
....TH
......
......
s.....

== D 1 ==

......
....T.
.....H
......
s.....

== L 5 ==

......
....T.
....H.
......
s.....

......
....T.
...H..
......
s.....

......
......
..HT..
......
s.....

......
......
.HT...
......
s.....

......
......
HT....
......
s.....

== R 2 ==

......
......
.H....  (H covers T)
......
s.....

......
......
.TH...
......
s.....

After simulating the rope, you can count up all of the positions the tail visited at least once. In this diagram, s again marks the starting position (which the tail also visited) and # marks other positions the tail visited:

```
..##..
...##.
.####.
....#.
s###..
```

So, there are 13 positions the tail visited at least once.

Simulate your complete hypothetical series of motions. How many positions does the tail of the rope visit at least once?

In [37]:
#@title import file function

def import_directions(filename : str):
    directions = []
    with open(filename) as f:
        for line in f:
            direction, steps = line.rstrip('\n').split(' ')
            directions.append([direction, int(steps)])
    
    return directions

In [38]:
# @title test file import

import unittest

class TestProcessInput(unittest.TestCase):

    def test_import_directions(self):
        expected = [
            ['R', 2],
            ['D', 2],
            ['R', 2],
            ['U', 1],
            ['R', 2],
            ['L', 2],
            ['D', 1],
        ]
        self.assertEqual(import_directions('test.txt'), expected)

unittest.main(argv=[''], verbosity=2, exit=False)

test_follow_directions_for_longer_rope_simplest (__main__.TestProcessDirections.test_follow_directions_for_longer_rope_simplest) ... FAIL
test_import_directions (__main__.TestProcessInput.test_import_directions) ... ok

FAIL: test_follow_directions_for_longer_rope_simplest (__main__.TestProcessDirections.test_follow_directions_for_longer_rope_simplest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/_n/60jwg_ln5k78bwvr4p4x1sn40000gn/T/ipykernel_12616/1585975174.py", line 18, in test_follow_directions_for_longer_rope_simplest
    self.assertEqual(follow_directions_for_longer_rope(input, 1), 13)
AssertionError: 5 != 13

----------------------------------------------------------------------
Ran 2 tests in 0.004s

FAILED (failures=1)


head: 1,0 tail: 0,0
positions_tail_has_been: {'0,0'}
head: 2,0 tail: 1,0
positions_tail_has_been: {'0,0'}
head: 3,0 tail: 2,0
positions_tail_has_been: {'1,1', '0,0'}
head: 4,0 tail: 3,0
positions_tail_has_been: {'2,2', '1,1', '0,0'}
head: 4,1 tail: 3,0
positions_tail_has_been: {'2,2', '1,1', '0,0', '3,3'}
head: 4,2 tail: 4,1
positions_tail_has_been: {'2,2', '1,1', '0,0', '3,3'}
head: 4,3 tail: 4,2
positions_tail_has_been: {'2,2', '0,0', '3,3', '4,4', '1,1'}
head: 4,4 tail: 4,3
positions_tail_has_been: {'2,2', '0,0', '3,3', '4,4', '1,1'}
head: 3,4 tail: 4,3
positions_tail_has_been: {'2,2', '0,0', '3,3', '4,4', '1,1'}
head: 2,4 tail: 3,4
positions_tail_has_been: {'2,2', '0,0', '3,3', '4,4', '1,1'}
head: 1,4 tail: 2,4
positions_tail_has_been: {'2,2', '0,0', '3,3', '4,4', '1,1'}
head: 1,3 tail: 2,4
positions_tail_has_been: {'2,2', '0,0', '3,3', '4,4', '1,1'}
head: 2,3 tail: 2,4
positions_tail_has_been: {'2,2', '0,0', '3,3', '4,4', '1,1'}
head: 3,3 tail: 2,4
positions_tail_has_been: {'2,2',

In [39]:
def follow_directions(directions):
    tail_curr_x=0
    tail_curr_y=0
    head_curr_x=0
    head_curr_y=0
    # set containing all tail locations in format 'x,y'
    positions_tail_has_been=set() 
    for direction in directions:
        positions_tail_has_been.add(f'{tail_curr_x},{tail_curr_y}')
        for _ in range(direction[1]):
            match direction[0]:
                case 'U':
                    head_curr_y +=  1
                    if head_curr_y > tail_curr_y + 1:
                        tail_curr_y = head_curr_y - 1
                        tail_curr_x = head_curr_x
                    positions_tail_has_been.add(f'{tail_curr_x},{tail_curr_y}')
                case 'D':
                    head_curr_y -=  1
                    if head_curr_y < tail_curr_y - 1:
                        tail_curr_y = head_curr_y + 1
                        tail_curr_x = head_curr_x
                    positions_tail_has_been.add(f'{tail_curr_x},{tail_curr_y}')
                case 'R':
                    head_curr_x +=  1
                    if head_curr_x > tail_curr_x + 1:
                        tail_curr_x = head_curr_x - 1
                        tail_curr_y = head_curr_y
                    positions_tail_has_been.add(f'{tail_curr_x},{tail_curr_y}')
                case 'L':
                    head_curr_x -=  1
                    if head_curr_x < tail_curr_x - 1:
                        tail_curr_x = head_curr_x + 1
                        tail_curr_y = head_curr_y
                    positions_tail_has_been.add(f'{tail_curr_x},{tail_curr_y}')
            # print(f'SIMPLE head: {head_curr_x},{head_curr_y} tail: {tail_curr_x},{tail_curr_y}') 
            # print(f'SIMPLE positions_tail_has_been: {positions_tail_has_been}') 
            
    return len(positions_tail_has_been)    

In [40]:
# @title test follow_directions

import unittest

class TestProcessDirections(unittest.TestCase):

    def test_follow_directions(self):
        input = [
            ['R', 4],
            ['U', 4],
            ['L', 3],
            ['D', 1],
            ['R', 4],
            ['D', 1],
            ['L', 5],
            ['R', 2],
        ]
        self.assertEqual(follow_directions(input), 13)

unittest.main(argv=[''], verbosity=2, exit=False)

test_follow_directions (__main__.TestProcessDirections.test_follow_directions) ... ok
test_import_directions (__main__.TestProcessInput.test_import_directions) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


In [41]:
input_data = import_directions('input.txt')
visited = follow_directions(input_data)
print(f'Visited {visited} spaces')

Visited 6026 spaces


Your puzzle answer was 6026.

The first half of this puzzle is complete! It provides one gold star: *

## Part Two

A rope snaps! Suddenly, the river is getting a lot closer than you remember. The bridge is still there, but some of the ropes that broke are now whipping toward you as you fall through the air!

The ropes are moving too quickly to grab; you only have a few seconds to choose how to arch your body to avoid being hit. Fortunately, your simulation can be extended to support longer ropes.

Rather than two knots, you now must simulate a rope consisting of ten knots. One knot is still the head of the rope and moves according to the series of motions. Each knot further down the rope follows the knot in front of it using the same rules as before.

Using the same series of motions as the above example, but with the knots marked H, 1, 2, ..., 9, the motions now occur as follows:

```
== Initial State ==

......
......
......
......
H.....  (H covers 1, 2, 3, 4, 5, 6, 7, 8, 9, s)

== R 4 ==

......
......
......
......
1H....  (1 covers 2, 3, 4, 5, 6, 7, 8, 9, s)

......
......
......
......
21H...  (2 covers 3, 4, 5, 6, 7, 8, 9, s)

......
......
......
......
321H..  (3 covers 4, 5, 6, 7, 8, 9, s)

......
......
......
......
4321H.  (4 covers 5, 6, 7, 8, 9, s)

== U 4 ==

......
......
......
....H.
4321..  (4 covers 5, 6, 7, 8, 9, s)

......
......
....H.
.4321.
5.....  (5 covers 6, 7, 8, 9, s)

......
....H.
....1.
.432..
5.....  (5 covers 6, 7, 8, 9, s)

....H.
....1.
..432.
.5....
6.....  (6 covers 7, 8, 9, s)

== L 3 ==

...H..
....1.
..432.
.5....
6.....  (6 covers 7, 8, 9, s)

..H1..
...2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

.H1...
...2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

== D 1 ==

..1...
.H.2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

== R 4 ==

..1...
..H2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

..1...
...H..  (H covers 2)
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

......
...1H.  (1 covers 2)
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

......
...21H
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

== D 1 ==

......
...21.
..43.H
.5....
6.....  (6 covers 7, 8, 9, s)

== L 5 ==

......
...21.
..43H.
.5....
6.....  (6 covers 7, 8, 9, s)

......
...21.
..4H..  (H covers 3)
.5....
6.....  (6 covers 7, 8, 9, s)

......
...2..
..H1..  (H covers 4; 1 covers 3)
.5....
6.....  (6 covers 7, 8, 9, s)

......
...2..
.H13..  (1 covers 4)
.5....
6.....  (6 covers 7, 8, 9, s)

......
......
H123..  (2 covers 4)
.5....
6.....  (6 covers 7, 8, 9, s)

== R 2 ==

......
......
.H23..  (H covers 1; 2 covers 4)
.5....
6.....  (6 covers 7, 8, 9, s)

......
......
.1H3..  (H covers 2, 4)
.5....
6.....  (6 covers 7, 8, 9, s)
```

Now, you need to keep track of the positions the new tail, 9, visits. In this example, the tail never moves, and so it only visits 1 position. However, be careful: more types of motion are possible than before, so you might want to visually compare your simulated rope to the one above.

Here's a larger example:

````
R 5
U 8
L 8
D 3
R 17
D 10
L 25
U 20
```

These motions occur as follows (individual steps are not shown):

```
== Initial State ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
...........H..............  (H covers 1, 2, 3, 4, 5, 6, 7, 8, 9, s)
..........................
..........................
..........................
..........................
..........................

== R 5 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
...........54321H.........  (5 covers 6, 7, 8, 9, s)
..........................
..........................
..........................
..........................
..........................

== U 8 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
................H.........
................1.........
................2.........
................3.........
...............54.........
..............6...........
.............7............
............8.............
...........9..............  (9 covers s)
..........................
..........................
..........................
..........................
..........................

== L 8 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
........H1234.............
............5.............
............6.............
............7.............
............8.............
............9.............
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
..........................

== D 3 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
.........2345.............
........1...6.............
........H...7.............
............8.............
............9.............
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
..........................

== R 17 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
................987654321H
..........................
..........................
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
..........................

== D 10 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
...........s.........98765
.........................4
.........................3
.........................2
.........................1
.........................H

== L 25 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
H123456789................

== U 20 ==

H.........................
1.........................
2.........................
3.........................
4.........................
5.........................
6.........................
7.........................
8.........................
9.........................
..........................
..........................
..........................
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
..........................
```

Now, the tail (9) visits 36 positions (including s) at least once:

```
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
#.........................
#.............###.........
#............#...#........
.#..........#.....#.......
..#..........#.....#......
...#........#.......#.....
....#......s.........#....
.....#..............#.....
......#............#......
.......#..........#.......
........#........#........
.........########.........
```

Simulate your complete series of motions on a larger rope with ten knots. How many positions does the tail of the rope visit at least once?

In [46]:
def follow_directions_for_longer_rope(directions, rope_length: int):
    # head is zero index, tail is rope_length - extra one for the tail
    positions=[[0,0] for _ in range(rope_length + 1)] 

    # set containing all tail locations in format 'x,y'
    positions_tail_has_been=set() 

    for direction in directions:     
        for _ in range(direction[1]):
            positions_tail_has_been.add(f'{positions[rope_length][0]},{positions[rope_length][1]}')
            for ip in range(len(positions)-1):
                # these temp vars are relative
                head_curr_y = positions[ip][1]
                head_curr_x = positions[ip][0]
                tail_curr_y = positions[ip+1][1]
                tail_curr_x = positions[ip+1][0]
                match direction[0]:
                    case 'U':
                        head_curr_y +=  1
                        if head_curr_y > tail_curr_y + 1:
                            tail_curr_y = head_curr_y - 1
                            tail_curr_x = head_curr_x
                    case 'D':
                        head_curr_y -=  1
                        if head_curr_y < tail_curr_y - 1:
                            tail_curr_y = head_curr_y + 1
                            tail_curr_x = head_curr_x
                    case 'R':
                        head_curr_x +=  1
                        if head_curr_x > tail_curr_x + 1:
                            tail_curr_x = head_curr_x - 1
                            tail_curr_y = head_curr_y
                    case 'L':
                        head_curr_x -=  1
                        if head_curr_x < tail_curr_x - 1:
                            tail_curr_x = head_curr_x + 1
                            tail_curr_y = head_curr_y

                positions[ip][1] = head_curr_y
                positions[ip][0] = head_curr_x
                positions[ip+1][1] = tail_curr_y
                positions[ip+1][0] = tail_curr_x

            print(f'head: {head_curr_x},{head_curr_y} tail: {tail_curr_x},{tail_curr_y}') 
            print(f'positions_tail_has_been: {positions_tail_has_been}') 
            
    return len(positions_tail_has_been)    

In [49]:
# @title test follow_directions_for_longer_rope

import unittest

class TestProcessDirections(unittest.TestCase):

    def test_follow_directions_for_longer_rope_simplest(self):
        input = [
            ['R', 4],
            ['U', 4],
            ['L', 3],
            ['D', 1],
            ['R', 4],
            ['D', 1],
            ['L', 5],
            ['R', 2],
        ]
        self.assertEqual(follow_directions_for_longer_rope(input, 1), 13)

    def test_follow_directions_for_longer_rope_harder(self):
        input = [
            ['R', 5],
            ['U', 8],
            ['L', 8],
            ['D', 3],
            ['R', 17],
            ['D', 10],
            ['L', 25],
            ['U', 20],
        ]
        self.assertEqual(follow_directions_for_longer_rope(input, 9), 36)

unittest.main(argv=[''], verbosity=2, exit=False)

test_follow_directions_for_longer_rope_harder (__main__.TestProcessDirections.test_follow_directions_for_longer_rope_harder) ... FAIL
test_follow_directions_for_longer_rope_simplest (__main__.TestProcessDirections.test_follow_directions_for_longer_rope_simplest) ... ok
test_import_directions (__main__.TestProcessInput.test_import_directions) ... ok

FAIL: test_follow_directions_for_longer_rope_harder (__main__.TestProcessDirections.test_follow_directions_for_longer_rope_harder)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/_n/60jwg_ln5k78bwvr4p4x1sn40000gn/T/ipykernel_12616/2423440685.py", line 31, in test_follow_directions_for_longer_rope_harder
    self.assertEqual(follow_directions_for_longer_rope(input, 9), 13)
AssertionError: 87 != 13

----------------------------------------------------------------------
Ran 3 tests in 0.007s

FAILED (failures=1)


head: 1,0 tail: 0,0
positions_tail_has_been: {'0,0'}
head: 2,0 tail: 1,0
positions_tail_has_been: {'0,0'}
head: 3,0 tail: 2,0
positions_tail_has_been: {'1,0', '0,0'}
head: 4,0 tail: 3,0
positions_tail_has_been: {'1,0', '0,0', '2,0'}
head: 5,0 tail: 4,0
positions_tail_has_been: {'1,0', '0,0', '2,0', '3,0'}
head: 5,1 tail: 4,0
positions_tail_has_been: {'0,0', '1,0', '4,0', '3,0', '2,0'}
head: 5,2 tail: 5,1
positions_tail_has_been: {'0,0', '1,0', '4,0', '3,0', '2,0'}
head: 5,3 tail: 5,2
positions_tail_has_been: {'0,0', '5,1', '1,0', '4,0', '3,0', '2,0'}
head: 5,4 tail: 5,3
positions_tail_has_been: {'5,2', '0,0', '5,1', '1,0', '4,0', '3,0', '2,0'}
head: 5,5 tail: 5,4
positions_tail_has_been: {'5,2', '0,0', '5,1', '5,3', '1,0', '4,0', '3,0', '2,0'}
head: 5,6 tail: 5,5
positions_tail_has_been: {'5,2', '0,0', '5,1', '5,3', '5,4', '1,0', '4,0', '3,0', '2,0'}
head: 5,7 tail: 5,6
positions_tail_has_been: {'5,2', '0,0', '5,1', '5,3', '5,4', '1,0', '4,0', '3,0', '2,0', '5,5'}
head: 5,8 tail: 5,7
p